## Implementing K-means Clustering From Scratch

## Libraries to Import

In [32]:
import urllib
import random
import numpy as np
import pandas as pd
from math import sqrt, floor
from urllib.request import urlretrieve

## Read data from url and convert to dataframe

In [8]:
iris = 'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
urlretrieve(iris)

df=pd.read_csv(iris,sep=',')

In [9]:
df.head()

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


## Assign column names to the table

In [14]:
attributes = ["sepal_length","sepal_width","petal_length","petal_width","class"]
df.columns = attributes
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [51]:
df.dtypes

sepal_length    float64
sepal_width     float64
petal_length    float64
petal_width     float64
class            object
dtype: object

## Feature Set

In [109]:
X = df.loc[:, df.columns != 'class']
X.head()

,sepal_length,sepal_width,petal_length,petal_width
0,4.9,3.0,1.4,0.2
1,4.7,3.2,1.3,0.2
2,4.6,3.1,1.5,0.2
3,5.0,3.6,1.4,0.2
4,5.4,3.9,1.7,0.4


## Converting to feature Vector

In [110]:
iris_ds = X.as_matrix()

## Random Initialisation of centroids

In [111]:
def initialize(ds, k):

    # Number of attributes in dataset
    n = np.shape(ds)[1]
    
    # The centroids
    centroids = np.mat(np.zeros((k,n)))

    # Create random centroids (get min, max attribute values, randomize in that range)
    for j in range(n):
        min_j = min(ds[:,j])
        range_j = float(max(ds[:,j]) - min_j)
        centroids[:,j] = min_j + range_j * np.random.rand(k, 1)

    # Return centroids as numpy array
    return centroids

In [112]:
initialize(ds = iris_ds,k =3)

matrix([[7.8049922 , 3.24991134, 3.87132496, 0.54113208],
        [4.75909528, 3.44224528, 6.07771388, 1.96235823],
        [5.17253204, 2.93342755, 3.98011672, 0.12740564]])

## Computing Eucledian Distance

In [113]:
def euclidean_dist(A, B):
    return np.sqrt(np.sum((A - B)**2))

In [116]:
import scipy.spatial.distance as metric

def euclidean_dist(A, B):
    
    #Calculate Euclidean distance between 2 n-dimension points
    return metric.euclidean(A, B)

## K-means Clustering

In [117]:
def cluster(ds, k):
    
    # Number of rows in dataset
    m = np.shape(ds)[0]
    
    # Hold the instance cluster assignments
    cluster_assignments = np.mat(np.zeros((m, 2)))
    
    # Initialize centroids
    cents = initialize(ds, k)
    
    # Preserve original centroids
    cents_orig = cents.copy()
    
    changed = True
    num_iter = 0
    
    # Loop until no changes to cluster assignments
    while changed:

        changed = False
        
        # For every instance (row in dataset)
        for i in range(m):

            # Track minimum distance, and vector index of associated cluster
            min_dist = np.inf
            min_index = -1
            
            # Calculate distances
            for j in range(k):

                dist_ji = euclidean_dist(cents[j,:], ds[i,:])
                if dist_ji < min_dist:
                    min_dist = dist_ji
                    min_index = j
            
            # Check if cluster assignment of instance has changed
            if cluster_assignments[i, 0] != min_index: 
                changed = True
                
                # Assign instance to appropriate cluster
                cluster_assignments[i, :] = min_index, min_dist**2
                
        # Update centroid location
        for cent in range(k):
            points = ds[np.nonzero(cluster_assignments[:,0].A==cent)[0]]
            cents[cent,:] = np.mean(points, axis=0)

        # Count iterations
        num_iter += 1

    # Return important stuff when done
    return cents, cluster_assignments, num_iter, cents_orig

## Testing the k-means

In [120]:
centroids, cluster_assignments, iters, orig_centroids = cluster(iris_ds, 3)
print('Number of iterations:', iters)
print('\nFinal centroids:\n', centroids)
print('\nCluster membership and error of first 10 instances:\n', cluster_assignments[:10])
print('\nOriginal centroids:\n', orig_centroids)

Number of iterations: 6

Final centroids:
 [[5.00408163 3.41632653 1.46530612 0.24489796]
 [6.85       3.07368421 5.74210526 2.07105263]
 [5.9016129  2.7483871  4.39354839 1.43387097]]

Cluster membership and error of first 10 instances:
 [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

Original centroids:
 [[4.97269511 2.10579952 3.22971728 1.47076946]
 [5.8215289  4.08840135 6.47693413 0.70832954]
 [5.35785201 2.19439183 5.66579457 0.79252956]]
